In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import learning_curve
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv("raw_data/20231128_only_deaths_ep_weights.csv")

In [4]:
df = df[df['allegiance'].map(df['allegiance'].value_counts()) > 30]

In [7]:
df = df[df['killer'].map(df['killer'].value_counts()) > 30]

In [8]:
df = df[df['killers_house'].map(df['killers_house'].value_counts()) > 30]

In [9]:
df = df[df['location'].map(df['location'].value_counts()) > 30]

In [10]:
df = df[df['method'].map(df['method'].value_counts()) > 30]

In [11]:
df

,Unnamed: 0,allegiance,killer,killers_house,location,method,name,episode_global_num,weights_simple
45,45,Smallfolk,Arya Stark,House Stark,King's Landing,Sword,Stableboy,8,0.999550
82,82,Free Folk,Night's Watch brother,Night's Watch,Beyond the Wall,Arrow,Wildling,16,0.952338
149,149,House Baratheon of Dragonstone,Baratheon of King's Landing soldier,House Baratheon of King's Landing,King's Landing,Sword,Baratheon of Dragonstone soldier,19,0.973921
151,151,House Baratheon of Dragonstone,Baratheon of King's Landing soldier,House Baratheon of King's Landing,King's Landing,Sword,Baratheon of Dragonstone soldier,19,0.973921
164,164,House Baratheon of Dragonstone,Baratheon of King's Landing soldier,House Baratheon of King's Landing,King's Landing,Sword,Baratheon of Dragonstone soldier,19,0.973921
...,...,...,...,...,...,...,...,...,...
2218,2218,None,Daenerys Targaryen,House Targaryen,King's Landing,Dragonfire (Dragon),King's Landing Citizen,72,0.895234
2219,2219,None,Daenerys Targaryen,House Targaryen,King's Landing,Dragonfire (Dragon),King's Landing Citizen,72,0.895234
2220,2220,None,Daenerys Targaryen,House Targaryen,King's Landing,Dragonfire (Dragon),King's Landing Citizen,72,0.895234
2221,2221,None,Daenerys Targaryen,House Targaryen,King's Landing,Dragonfire (Dragon),King's Landing Citizen,72,0.895234


In [12]:
df = df.drop(columns=["Unnamed: 0",'name'], axis=1)

In [13]:
y = df["episode_global_num"]
y = y.to_frame(name="episode_global_num")
y.head()

,episode_global_num
45,8
82,16
149,19
151,19
164,19


In [14]:
X = df
X = X.drop(columns = ["episode_global_num"], axis=1)
X.head()

,allegiance,killer,killers_house,location,method,weights_simple
45,Smallfolk,Arya Stark,House Stark,King's Landing,Sword,0.999550
82,Free Folk,Night's Watch brother,Night's Watch,Beyond the Wall,Arrow,0.952338
149,House Baratheon of Dragonstone,Baratheon of King's Landing soldier,House Baratheon of King's Landing,King's Landing,Sword,0.973921
151,House Baratheon of Dragonstone,Baratheon of King's Landing soldier,House Baratheon of King's Landing,King's Landing,Sword,0.973921
164,House Baratheon of Dragonstone,Baratheon of King's Landing soldier,House Baratheon of King's Landing,King's Landing,Sword,0.973921


In [26]:
num_transformer = Pipeline([('standard_scaler', StandardScaler())])

cat_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

preprocessor = ColumnTransformer([
    ('num_transformer', num_transformer, ["weights_simple"]),
    ('cat_transformer', cat_transformer, ['allegiance','killer','killers_house','location','method'])
])

In [27]:
X_train = pd.DataFrame(preprocessor.fit_transform(X_train),
            columns=preprocessor.get_feature_names_out())

ValueError: A given column is not a column of the dataframe

In [28]:
X_train.columns

Index(['num_transformer__weights_simple',
       'cat_transformer__allegiance_Faith Militant',
       'cat_transformer__allegiance_Free Folk',
       'cat_transformer__allegiance_House Baratheon of Dragonstone',
       'cat_transformer__allegiance_House Bolton',
       'cat_transformer__allegiance_House Frey',
       'cat_transformer__allegiance_House Greyjoy',
       'cat_transformer__allegiance_House Lannister',
       'cat_transformer__allegiance_House Stark',
       'cat_transformer__allegiance_House Targaryen',
       'cat_transformer__allegiance_Night's Watch',
       'cat_transformer__allegiance_None',
       'cat_transformer__allegiance_Smallfolk',
       'cat_transformer__allegiance_Sons of the Harpy',
       'cat_transformer__killer_Arya Stark',
       'cat_transformer__killer_Baratheon of King's Landing soldier',
       'cat_transformer__killer_Bolton soldier',
       'cat_transformer__killer_Cersei Lannister',
       'cat_transformer__killer_Daenerys Targaryen',
       'cat

In [31]:
model = LinearRegression()
model.fit(X_train,y_train)
cv_results = cross_validate(model, X_train, y_train, cv=5, scoring=('r2', 'neg_mean_squared_error'))
test = cv_results["test_r2"].mean()
test

-6.839459326519043e+21